In [1]:
import pandas as pd
import torch


In [2]:
df = pd.read_csv('/content/Churn_Modelling.csv')
df.head()

,RowNumber,CustomerId,Surname,CreditScore,Geography,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Exited
0,1,15634602,Hargrave,619,France,Female,42,2,0.00,1,1,1,101348.88,1
1,2,15647311,Hill,608,Spain,Female,41,1,83807.86,1,0,1,112542.58,0
2,3,15619304,Onio,502,France,Female,42,8,159660.80,3,1,0,113931.57,1
3,4,15701354,Boni,699,France,Female,39,1,0.00,2,0,0,93826.63,0
4,5,15737888,Mitchell,850,Spain,Female,43,2,125510.82,1,1,1,79084.10,0


In [3]:
df = df.drop(columns = ['CustomerId' , 'Surname' , 'RowNumber'])
df.head()

,CreditScore,Geography,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Exited
0,619,France,Female,42,2,0.00,1,1,1,101348.88,1
1,608,Spain,Female,41,1,83807.86,1,0,1,112542.58,0
2,502,France,Female,42,8,159660.80,3,1,0,113931.57,1
3,699,France,Female,39,1,0.00,2,0,0,93826.63,0
4,850,Spain,Female,43,2,125510.82,1,1,1,79084.10,0


In [4]:
df['Geography'].unique()

array(['France', 'Spain', 'Germany'], dtype=object)

In [5]:
df['Gender'] = df['Gender'].map({'Female': 0, 'Male': 1})
df['Geography'] = df['Geography'].map({'France': 0, 'Germany': 1, 'Spain': 2})
df.head()

,CreditScore,Geography,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Exited
0,619,0,0,42,2,0.00,1,1,1,101348.88,1
1,608,2,0,41,1,83807.86,1,0,1,112542.58,0
2,502,0,0,42,8,159660.80,3,1,0,113931.57,1
3,699,0,0,39,1,0.00,2,0,0,93826.63,0
4,850,2,0,43,2,125510.82,1,1,1,79084.10,0


In [6]:
X = df.drop(columns = ['Exited'] , axis = 1)
y = df['Exited']

In [7]:
import torch.optim as optim
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from torch.utils.data import DataLoader, TensorDataset



X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)


In [8]:
# Scale the data
scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)


In [9]:
X_train = torch.tensor(X_train, dtype=torch.float32)
X_test = torch.tensor(X_test, dtype=torch.float32)
y_train = torch.tensor(y_train.values, dtype=torch.float32).view(-1, 1)
y_test = torch.tensor(y_test.values, dtype=torch.float32).view(-1, 1)


In [10]:
import torch.nn as nn
class SimpleNN(nn.Module):
    def __init__(self, input_dim):
        super(SimpleNN, self).__init__()
        self.fc1 = nn.Linear(input_dim, 32)
        self.fc2 = nn.Linear(32, 16)
        self.fc3 = nn.Linear(16, 1)
        self.relu = nn.ReLU()
        self.sigmoid = nn.Sigmoid()

    def forward(self, x):
        x = self.relu(self.fc1(x))
        x = self.relu(self.fc2(x))
        x = self.sigmoid(self.fc3(x))
        return x

# Initialize the model, criterion, and optimizer
input_dim = X_train.shape[1]
model = SimpleNN(input_dim)
criterion = nn.BCELoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)


In [11]:
# Train the model

train_dataset = TensorDataset(X_train, y_train)
train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True) # Adjust batch_size as needed




num_epochs = 100
for epoch in range(num_epochs):
    model.train()
    train_correct = 0
    train_total = 0
    for inputs, targets in train_loader:
        # Forward pass
        outputs = model(inputs)
        loss = criterion(outputs, targets)

        # Backward pass and optimization
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        # Calculate training accuracy
        predicted = outputs.round()
        train_correct += (predicted.eq(targets).sum().item())
        train_total += targets.size(0)

    train_accuracy = train_correct / train_total
    if (epoch+1) % 10 == 0:
        print(f'Epoch [{epoch+1}/{num_epochs}], Loss: {loss.item():.4f}, Training Accuracy: {train_accuracy:.4f}')


Epoch [10/100], Loss: 0.4541, Training Accuracy: 0.8591
Epoch [20/100], Loss: 0.2161, Training Accuracy: 0.8630
Epoch [30/100], Loss: 0.2130, Training Accuracy: 0.8676
Epoch [40/100], Loss: 0.3242, Training Accuracy: 0.8684
Epoch [50/100], Loss: 0.1914, Training Accuracy: 0.8708
Epoch [60/100], Loss: 0.3058, Training Accuracy: 0.8729
Epoch [70/100], Loss: 0.2376, Training Accuracy: 0.8714
Epoch [80/100], Loss: 0.4998, Training Accuracy: 0.8738
Epoch [90/100], Loss: 0.3380, Training Accuracy: 0.8728
Epoch [100/100], Loss: 0.5151, Training Accuracy: 0.8739


In [12]:
# Evaluate the model
model.eval()
with torch.no_grad():
    # Training accuracy
    train_outputs = model(X_train)
    train_predicted = train_outputs.round()
    train_accuracy = (train_predicted.eq(y_train).sum() / float(y_train.shape[0])).item()

    # Test accuracy
    test_outputs = model(X_test)
    test_predicted = test_outputs.round()
    test_accuracy = (test_predicted.eq(y_test).sum() / float(y_test.shape[0])).item()

    print(f'Training Accuracy: {train_accuracy:.4f}')
    print(f'Test Accuracy: {test_accuracy:.4f}')


Training Accuracy: 0.8731
Test Accuracy: 0.8600


In [13]:
new_data = {
    'CreditScore': 650,
    'Geography': 'France',
    'Gender': 'Male',
    'Age': 40,
    'Tenure': 5,
    'Balance': 60000.00,
    'NumOfProducts': 2,
    'HasCrCard': 1,
    'IsActiveMember': 1,
    'EstimatedSalary': 50000.00
}

new_df = pd.DataFrame([new_data])
new_df['Gender'] = new_df['Gender'].map({'Female': 0, 'Male': 1})
new_df['Geography'] = new_df['Geography'].map({'France': 0, 'Germany': 1, 'Spain': 2})

X_new = new_df.values
X_new = scaler.transform(X_new)
X_new_tensor = torch.tensor(X_new, dtype=torch.float32)

with torch.no_grad():
    new_prediction = model(X_new_tensor)
    predicted_value = new_prediction.round().item()

print(f'Predicted value for the new data: {predicted_value}')

Predicted value for the new data: 0.0


/usr/local/lib/python3.10/dist-packages/sklearn/base.py:465: UserWarning: X does not have valid feature names, but StandardScaler was fitted with feature names
  warnings.warn(
